# dwave-neal

- [SA](https://qiita.com/ShataKurashi/items/c0c6044e97fa9e4a9471#:~:text=%E7%84%BC%E3%81%8D%E3%81%AA%E3%81%BE%E3%81%97%E6%B3%95%EF%BC%88%E3%82%84%E3%81%8D%E3%81%AA%E3%81%BE%E3%81%97,%E3%81%A6%E3%80%81%E3%82%88%E3%81%84%E8%BF%91%E4%BC%BC%E3%82%92%E4%B8%8E%E3%81%88%E3%82%8B%E3%80%82)
- [組み合わせ最適問題に対するSA法　中野秀男・中西義郎](http://www.orsj.or.jp/~archive/pdf/bul/Vol.31_01_043.pdf)

In [20]:
#!pip3 install pandas

In [21]:
#!pip3 install scipy

In [22]:
#!pip3 install dwave-neal

In [23]:
#!pip3 install pyqubo

In [24]:
#!pip3 install matplotlib

In [25]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random
import matplotlib.pyplot as plt
import timeit

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [26]:
ost40 = pd.read_csv("../../input/ost40.csv", sep=',', index_col=0)
ost40.shape

(40, 4)

In [27]:
def make_t_list_columns_num_samples(df):
    t_list = []
    t_list.append(sum(df['Y']))
    t_list.append(np.dot(df['Y'], df['LI']))
    t_list.append(np.dot(df['Y'], df['SEX']))
    t_list.append(np.dot(df['Y'], df['AOP']))
    
    columns = list(df.columns)
    num_samples = df.shape[0]
    
    return t_list, columns, num_samples 

In [28]:
t_list, columns, num_samples = make_t_list_columns_num_samples(ost40)

### D-wave neal

In [29]:
def neal_sampling(df, num_reads):
    valid_y_list = {}
    valid_y_num = {}
    for t1 in range(0, df.shape[0]+1):
        y = Array.create('y', shape=df.shape[0], vartype='BINARY')

        #QUBO式で定式化
        H = (sum(y) - t_list[0])**2
        H += (np.dot(df[columns[1]], y) - t1)**2
        for j in range(2, len(columns)):
            H_plus = (np.dot(df[columns[j]], y) - t_list[j])**2
            H += H_plus

        qb = H.compile().to_qubo()
        sa_sampler = SimulatedAnnealingSampler()
        res = sa_sampler.sample_qubo(qb[0], num_reads=num_reads)

        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        for y_info in list(res.record):
            if sum(y_info[0]) == t_list[0]:#int同士の比較
                y = pd.Series(y_info[0])
                if np.dot(df[columns[1]], y) == t1:#int同士の比較
                    for j in range(2, len(columns)):
                        if np.dot(df[columns[j]], y) != t_list[j]:#int同士の比較
                            break
                    else:
                        if all(list(y_info[0]) != p for p in valid_y_list[t1]):
                            valid_y_list[t1].append(list(y_info[0]))
                            valid_y_num[t1] += 1
                            print('perfect')
    return valid_y_list, valid_y_num

In [19]:
valid_y_list, valid_y_num = neal_sampling(ost40, 100)
print(valid_y_list)
print('--------')
print(valid_y_num)

KeyboardInterrupt: 

In [30]:
loop = 100
df = ost40
num_reads = 100
result = timeit.timeit('neal_sampling(df, num_reads)', globals=globals(), number=loop)
print(result / loop, '秒')

13.297175053000101 秒


In [31]:
valid_y_list

NameError: name 'valid_y_list' is not defined

In [ ]:
valid_y_num

In [ ]:
valid_y_num[9]/sum(valid_y_num.values())